# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
import argparse
from itertools import product
import math
import nltk
from pathlib import Path
import os
import glob
import string

In [3]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def combine_files(input_folder, output_file):
    with open(output_file, 'w', encoding='utf-8') as combined_file:
        folders = ['unsup', 'pos', 'neg']
        for folder in folders:
            folder_path = os.path.join(input_folder, folder)
            files = glob.glob(os.path.join(folder_path, '*.txt'))

            for file in files:
                with open(file, 'r', encoding='utf-8') as current_file:
                    content = current_file.read()
                    sentences = [remove_punctuation(sentence.strip()) for sentence in content.split('.') if sentence.strip()]
                    combined_file.write('\n'.join(sentences) + '\n')
 
input_folder = '/Users/damirabdulaev/Downloads/aclImdb/train'
output_file = '/Users/damirabdulaev/Downloads/aclImdb/train/combined.txt'

combine_files(input_folder, output_file)

In [4]:
def load_train_data(train_path, percent=50):
    with open(train_path, 'r') as f:
        train = [l.strip() for l in f.readlines()]
    total_samples = len(train)
    num_samples_to_load = int(total_samples * percent / 100)
    return train[:num_samples_to_load]

In [5]:
train_file = '/Users/damirabdulaev/Downloads/aclImdb/train/combined.txt'

In [6]:
train = load_train_data(train_file)

In [7]:
SOS = '<s> '
EOS = '</s> '
UNK = '<UNK>'

def end_and_start_tokens(sentences, n):
    if n > 1:
        sos = SOS * (n-1)
    else:
        sos = SOS

    return [f'{sos}{s} {EOS}' for s in sentences]

def replace(tokens):
    vocab = nltk.FreqDist(tokens)
    return [token if vocab[token] >= 1 else UNK for token in tokens]

def preprocess(sentences, n):
    sentences = end_and_start_tokens(sentences, n)
    tokens = ' '.join(sentences).split(' ')
    tokens = replace(tokens)
    return tokens

In [8]:
print(preprocess(train[:2], 5))

['<s>', '<s>', '<s>', '<s>', 'A', 'newspaperman', 'Johnny', 'Twennies', 'living', 'in', 'the', '90s', 'with', 'a', 'complete', '20s', 'personality', 'and', 'lifestyle', '', 'fedora', 'manual', 'typewriter', 'the', 'Charleston', 'the', 'works', '</s>', '', '<s>', '<s>', '<s>', '<s>', 'Its', 'a', 'great', 'idea', 'for', 'a', 'movie', 'and', 'it', 'couldnt', 'have', 'been', 'done', 'better', '</s>', '']


In [9]:
class LanguageModel(object):
    def __init__(self, train_data, n, laplace=1):
        self.n = n
        self.laplace = laplace
        self.tokens = preprocess(train_data, n)
        self.vocab  = nltk.FreqDist(self.tokens)
        self.model  = self._create_model()

    def _smooth(self):
        vocab_size = len(self.vocab)

        n_grams = nltk.ngrams(self.tokens, self.n)
        n_vocab = nltk.FreqDist(n_grams)

        m_grams = nltk.ngrams(self.tokens, self.n-1)
        m_vocab = nltk.FreqDist(m_grams)

        def smoothed_count(n_gram, n_count):
            m_gram = n_gram[:-1]
            m_count = m_vocab[m_gram]
            return (n_count + self.laplace) / (m_count + self.laplace * vocab_size)

        return { n_gram: smoothed_count(n_gram, count) for n_gram, count in n_vocab.items() }
    
    def _create_model(self):
        if self.n == 1:
            num_tokens = len(self.tokens)
            return { (unigram,): count / num_tokens for unigram, count in self.vocab.items() }
        else:
            return self._smooth()

    def _best_candidate(self, prev):
        candidates = ((ngram[-1],prob) for ngram,prob in self.model.items() if ngram[:-1]==prev)
        candidates = sorted(candidates, key=lambda candidate: candidate[1], reverse=True)
        if len(candidates) == 0:
            return "</s>", 1
        else:
            return candidates[0]

In [10]:
n = 5
lm5 = LanguageModel(train, n)
lm4 = LanguageModel(train, n-1)
lm3 = LanguageModel(train, n-2)
lm2 = LanguageModel(train, n-3)

In [11]:
def split_and_give_predictions(tokens, lm, n):
    best_candidates = []

    for i in range(len(tokens) - (n - 1)):
        context = tuple(tokens[i:i+(n - 1)])
    
        # Get the best candidate for the current context
        candidate, probability = lm._best_candidate(context)

        print(context, candidate, probability)
    
        # Add the best candidate to the list
        best_candidates.append((candidate, probability))
    
    # Print the best candidates
    for candidate, probability in best_candidates:
        print(f"Best candidate: {candidate}, Probability: {probability}")

In [12]:
splitted_text = "Fox was the perfect".split()

# Get the best candidate for the current context
def stupid_backoff(context, models):
    for i in range(1, len(models) + 1):
        lm = models[i - 1]
        if i != 1:
            context = context[1:]
        candidate, probability = lm._best_candidate(tuple(context))
        if candidate != "</s>":
            return candidate, probability
    return "</s>", 1

models = [lm5, lm4, lm3, lm2]
best_candidate, best_probability = stupid_backoff(splitted_text, models)
print(f"Best candidate: {best_candidate}, Probability: {best_probability}")

Best candidate: choice, Probability: 2.4359942510535674e-05


In [13]:
def load_test_data(test_path, percent=1):
    with open(test_path, 'r') as f:
        test = [l.strip() for l in f.readlines()]
    total_samples = len(test)
    num_samples_to_load = int(total_samples * percent / 100)
    return test[-num_samples_to_load:]

In [14]:
test = load_test_data("/Users/damirabdulaev/Downloads/aclImdb/train/combined.txt")

In [15]:
import random

def introduce_mistakes(test_data, probability, n):
    # Iterate over each sentence in the test data
    sentences = []
    for sentence in test_data:
        # Split the sentence into words
        words = sentence.split()

        # Determine the number of words to introduce mistakes into
        num_words_to_modify = max(1, round(len(words) * probability))
        
        # Randomly select words to modify
        if len(words) == 0:
            continue
        words_to_modify = random.sample(words, num_words_to_modify)

        # Introduce mistakes into selected words
        for word in words_to_modify:
            # Randomly select positions to modify within the word
            positions_to_modify = random.sample(range(len(word)), min(n, len(word)))

            # Introduce mistakes at selected positions
            modified_word = list(word)
            for pos in positions_to_modify:
                # Introduce a random misspelled letter at the selected position
                modified_word[pos] = random.choice([char for char in 'abcdefghijklmnopqrstuvwxyz' if char != word[pos].lower()])
            modified_word = ''.join(modified_word)

            # Replace the original word with the modified word in the sentence
            sentence = sentence.replace(word, modified_word)
            
            sentences.append(sentence)

    return sentences

mistaken_sentences = introduce_mistakes(test, probability=0.01, n=2)

In [16]:
mistaken_sentences[0]

'horror flick fall backs gore pointless nudity kkocvs against the catholic church'

In [19]:
def replace_with_best_candidate(sentence, models):
    # Split the sentence into words
    words = sentence.split()

    # Iterate over each word in the sentence
    for i, word in enumerate(words):
        # Check if the word is not in the vocabulary
        if word not in models[0].vocab:
            # Create the context for the current word
            
            context = words[:i]
            print(context)

            # Get the best candidate for the current context
            best_candidate, _ = stupid_backoff(context, models)

            # Replace the original word with the best candidate
            words[i] = best_candidate

    # Join the words back into a sentence
    corrected_sentence = ' '.join(words)
    return corrected_sentence

# Example usage
test_sentences = [
    "Thsi is a tst sentence.",
    "Anohter example sentence.",
    "Yet antoher example."
]

# Iterate over each test sentence and replace mistaken words with best candidates
for sentence in test_sentences:
    corrected_sentence = replace_with_best_candidate(sentence, models)
    print("Original:", sentence)
    print("Corrected:", corrected_sentence)
    print()

['Thsi', 'is', 'a']
['Thsi', 'is', 'a', '</s>']
Original: Thsi is a tst sentence.
Corrected: Thsi is a </s> 

[]
['</s>', 'example']
Original: Anohter example sentence.
Corrected: </s> example </s>

['Yet']
['Yet', '</s>']
Original: Yet antoher example.
Corrected: Yet </s> </s>


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

*Your text here...*

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.